# TorchVision Tutorial Based Pipeline

Following TORCHVISION OBJECT DETECTION FINETUNING TUTORIAL at https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from pathlib import Path
import os
# Vlad's drive
# repo_path = Path.cwd()/'drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det'
# Joaquin's drive
repo_path = Path.cwd()/'drive/MyDrive/calcification_detection/calc-det/notebooks'
os.chdir(str(repo_path))

thispath = Path.cwd().resolve()
import sys; sys.path.insert(0, str(thispath.parent))

Copy extracted patches used for detection (224 size with stride 100) to the local machine. ***Needs to be done once per machine instantization***. 

~ 3 minutes to complete

In [3]:
# # Vlad's Drive
# # !cp -r /content/drive/MyDrive/calcification-detection-project/calcification_detecion/new_data_rois/data_rois.zip /home/
# # Joaquin's Drive
# !cp -r /content/drive/MyDrive/calcification_detection/rois_all_method/data_rois_224.zip /home/data_rois.zip
# !unzip /home/data_rois.zip -d /home
# !mv /home/home/vzalevskyi/projects/data_rois /home/data_rois
# !rm -r /home/home
# !pip install transformers

Now let's instantiate the model and the optimizer

And now let's train the model for 10 epochs, evaluating at the end of every epoch.

In [4]:
cfg = {
    'model': {
        'activation': 'LeakyReLU',
        'bloc_act': None,
        'use_middle_activation': True,
        'dropout': 0.4,
        'fc_dims': None,
        'freeze_weights': False,
        'backbone': 'net2',
        'pretrained': True,
        'checkpoint_path': None,
        'anchor_sizes': [12, 14],
        'anchor_ratios': [1.0],
        'num_classes': 2
    },
    'dataset': {
        'extract_patches': False,
        'delete_previous': False,
        'extract_patches_method': 'all',
        'patch_size': 224,
        'stride': 100,
        'min_breast_fraction_roi': 0.7,
        'n_jobs': -1,
        'cropped_imgs': True,
        'ignore_diameter_px': 15,
        'train_neg_to_pos_ratio': None,
        'balancing_seed': 0,
        'normalization': 'min_max',
        'get_lesion_bboxes': True,
        'for_detection_net': True,
        'patch_images_path': '/home/data_rois/',
        'detection_bbox_size':14
    },
    'dataloaders': {
        'train_batch_size': 8,
        'val_batch_size': 16
    },
    'data_aug': {
        'prob': 0
    },
    'training': {
        'criterion': 'BCEWithLogitsLoss',
        'optimizer': 'Adam',
        'optimizer_args': {
            'lr': 0.0001 #, 'momentum': 0.9
        },
        'lr_scheduler': 'StepLR',
        'lr_scheduler_args': {
            'step_size': 3, 'gamma': 0.1
        },
        'n_epochs': 30,
        'best_metric': 'f1_score',
        'resume_training': False,
        'early_stopping': True,
        'early_stopping_args':{
            'min_diff': 0.0001,
            'max_epoch': 3
        },
        'log_iters': 100,
        'max_iters_per_epoch': None
        },
    'experiment_name': '32_resnet_01'
}

import yaml

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)

In [5]:
from pathlib import Path
thispath = Path.cwd().resolve()
import sys; sys.path.insert(0, str(thispath.parent))

import logging
import torch
import torchvision
import yaml

import torch.nn as nn
import torch.optim as optim

from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

import deep_learning.dl_utils as dl_utils
from deep_learning.dataset.dataset import INBreast_Dataset_pytorch
from deep_learning.models.base_classifier import CNNClasssifier
from deep_learning.detection_models.train import train_model

logging.basicConfig(level=logging.INFO)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def main():
    # read the configuration file
    config_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
    with open(config_path, "r") as ymlfile:
        cfg = yaml.safe_load(ymlfile)

    # for colab it needs to be specified additionally
    dataset_arguments = cfg['dataset']

    # use the configuration for the dataset
    dataset_arguments = cfg['dataset']
    dataset_arguments['patch_images_path'] = Path(dataset_arguments['patch_images_path'])
    datasets = {
        'train': INBreast_Dataset_pytorch(
            partitions=['train'], neg_to_pos_ratio=dataset_arguments['train_neg_to_pos_ratio'],
            **dataset_arguments),
        'val': INBreast_Dataset_pytorch(
            partitions=['validation'], neg_to_pos_ratio=None, **dataset_arguments)
    }

    # use the configuration for the dataloaders
    def collate_fn(batch):
        return tuple(zip(*batch))
    dataloaders = {
        'val': DataLoader(
            datasets['val'], batch_size=cfg['dataloaders']['val_batch_size'],
            num_workers=4, collate_fn=collate_fn, drop_last=False),
        'train': DataLoader(
            datasets['train'], batch_size=cfg['dataloaders']['train_batch_size'],
            shuffle=True, collate_fn=collate_fn, num_workers=4, drop_last=False)
    }

    # model settings
    if cfg['model']['checkpoint_path'] is not None:
        model_ckpt = torch.load(cfg['model']['checkpoint_path'])
        model = dl_utils.get_model_from_checkpoint(model_ckpt, cfg['model']['freeze_weights'])
    else:
        model = CNNClasssifier(
            activation=getattr(nn, cfg['model']['activation'])(),
            dropout=cfg['model']['dropout'],
            fc_dims=cfg['model']['fc_dims'],
            freeze_weights=cfg['model']['freeze_weights'],
            backbone=cfg['model']['backbone'],
            pretrained=cfg['model']['pretrained'],
        )
        model = model.model

    modules = list(model.children())[:-2]      # delete the last fc layers.
    last_submodule_childs = list(modules[-1][-1].children())
    for i in range(len(last_submodule_childs)-1, -1, -1):
        if hasattr(last_submodule_childs[i], 'out_channels'):
            out_channels = last_submodule_childs[i].out_channels
            break
    model_backbone = nn.Sequential(*modules)
    model_backbone.out_channels = out_channels
    anchor_generator = AnchorGenerator(
        sizes=(cfg['model']['anchor_sizes'],),
        aspect_ratios=(cfg['model']['anchor_ratios'],))

    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0'], output_size=7, sampling_ratio=2)

    # put the pieces together inside a FasterRCNN model
    model = FasterRCNN(
        model_backbone,
        num_classes=cfg['model']['num_classes'],
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler,
        image_mean=[0., 0., 0.],
        image_std=[1., 1., 1.],
    )

    model.to(device)

    # training configs
    optimizer = getattr(optim, cfg['training']['optimizer'])
    optimizer = optimizer(model.parameters(), **cfg['training']['optimizer_args'])

    scheduler = getattr(lr_scheduler, cfg['training']['lr_scheduler'])
    scheduler = scheduler(optimizer, **cfg['training']['lr_scheduler_args'])

    # train the model
    train_model(datasets, dataloaders, model, optimizer, scheduler, cfg)

In [6]:
# cfg['model']['activation'] = 'LeakyReLU'
# cfg['model']['dropout'] = 0.4
# cfg['model']['fc_dims'] = None
cfg['model']['backbone'] = 'resnet50'
cfg['model']['pretrained'] = True
cfg['model']['checkpoint_path'] = '/content/drive/MyDrive/drive_vlad/deepl_runs/resnet50_05/resnet50_05.pt'
cfg['model']['anchor_sizes'] = [14]
cfg['model']['anchor_ratios'] = [1.0]
cfg['model']['num_classes'] = 2

cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['dataset']['normalization'] = 'z_score'

cfg['dataloaders']['train_batch_size'] = 6
cfg['dataloaders']['val_batch_size'] = 12

cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 3, 'gamma': 0.1}
cfg['training']['n_epochs'] = 15
cfg['training']['early_stopping'] = False
cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'AP_IoU_0.50_all'
cfg['training']['max_iters_per_epoch'] = 200

cfg['experiment_name'] = 'det_resnet50_our_02'

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:52: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/det_resnet50_our_02


creating index...
index created!
Epoch: [0]  [  0/200]  eta: 0:18:54  lr: 0.000000  loss: 1.5723 (1.5723)  loss_classifier: 0.6890 (0.6890)  loss_box_reg: 0.0009 (0.0009)  loss_objectness: 0.7923 (0.7923)  loss_rpn_box_reg: 0.0901 (0.0901)  time: 1.5569  data: 0.4887  max mem: 9834
Epoch: [0]  [100/200]  eta: 0:10:38  lr: 0.000014  loss: 0.1077 (0.3619)  loss_classifier: 0.0325 (0.1213)  loss_box_reg: 0.0080 (0.0070)  loss_objectness: 0.0516 (0.1959)  loss_rpn_box_reg: 0.0178 (0.0377)  time: 1.0057  data: 0.0199  max mem: 11723
Test:  [ 0/90]  eta: 0:01:57  model_time: 0.6386 (0.6386)  evaluator_time: 0.0157 (0.0157)  time: 1.3040  data: 0.6451  max mem: 11723
Test:  [89/90]  eta: 0:00:00  model_time: 0.6451 (0.6453)  evaluator_time: 0.0135 (0.0420)  time: 0.6984  data: 0.0402  max mem: 11723
Test: Total time: 0:01:07 (0.7459 s / it)
Averaged stats: model_time: 0.6451 (0.6453)  evaluator_time: 0.0135 (0.0420)
Accumulating evaluation results...


INFO:root:Train Loss: 0.2438, AP_IoU_0.50_all: 0.4197


DONE (t=0.24s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.420
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.042
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.142
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.252
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.310
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.310
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0998, AP_IoU_0.50_all: 0.6175


...

INFO:root:Train Loss: 0.0694, AP_IoU_0.50_all: 0.7131


DONE (t=0.14s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.410
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.713
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.439
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.410
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.249
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.477
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.543
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.543
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0637, AP_IoU_0.50_all: 0.7186


DONE (t=0.15s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.416
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.719
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.446
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.416
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.251
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.484
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.551
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.551
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000


INFO:root:----------
INFO:root:Training complete in 69m 51s
INFO:root:Best val AP_IoU_0.50_all: 0.720914, epoch 13


In [ ]:

# cfg['model']['activation'] = 'LeakyReLU'
# cfg['model']['dropout'] = 0.4
# cfg['model']['fc_dims'] = None
cfg['model']['backbone'] = 'resnet50'
cfg['model']['pretrained'] = True
cfg['model']['checkpoint_path'] = '/content/drive/MyDrive/drive_vlad/deepl_runs/resnet50_05/resnet50_05.pt'
cfg['model']['anchor_sizes'] = [12, 14, 16]
cfg['model']['anchor_ratios'] = [1.0]
cfg['model']['num_classes'] = 2

cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['dataset']['normalization'] = 'z_score'

cfg['dataloaders']['train_batch_size'] = 6
cfg['dataloaders']['val_batch_size'] = 12

cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 3, 'gamma': 0.1}
cfg['training']['n_epochs'] = 15
cfg['training']['early_stopping'] = False
cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'AP_IoU_0.50_all'
cfg['training']['max_iters_per_epoch'] = 200

cfg['experiment_name'] = 'det_resnet50_our_02'

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:52: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/det_resnet50_our_02


creating index...
index created!
Epoch: [0]  [  0/200]  eta: 0:20:00  lr: 0.000000  loss: 1.4300 (1.4300)  loss_classifier: 0.6333 (0.6333)  loss_box_reg: 0.0022 (0.0022)  loss_objectness: 0.7134 (0.7134)  loss_rpn_box_reg: 0.0810 (0.0810)  time: 1.6469  data: 0.5862  max mem: 11723
Epoch: [0]  [100/200]  eta: 0:10:46  lr: 0.000014  loss: 0.0887 (0.3528)  loss_classifier: 0.0324 (0.1150)  loss_box_reg: 0.0068 (0.0074)  loss_objectness: 0.0397 (0.1989)  loss_rpn_box_reg: 0.0127 (0.0315)  time: 1.0168  data: 0.0200  max mem: 11723
Test:  [ 0/90]  eta: 0:02:43  model_time: 1.0388 (1.0388)  evaluator_time: 0.0388 (0.0388)  time: 1.8113  data: 0.7301  max mem: 11723
Test:  [89/90]  eta: 0:00:00  model_time: 0.8107 (0.8180)  evaluator_time: 0.0290 (0.0750)  time: 0.8861  data: 0.0416  max mem: 11723
Test: Total time: 0:01:25 (0.9471 s / it)
Averaged stats: model_time: 0.8107 (0.8180)  evaluator_time: 0.0290 (0.0750)
Accumulating evaluation results...


INFO:root:Train Loss: 0.2304, AP_IoU_0.50_all: 0.2608


DONE (t=0.39s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.085
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.261
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.085
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.106
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.214
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.292
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.292
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0829, AP_IoU_0.50_all: 0.5993


...

DONE (t=0.23s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.400
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.705
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.424
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.400
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.241
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.475
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.547
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.547
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0551, AP_IoU_0.50_all: 0.7127


DONE (t=0.24s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.409
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.713
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.439
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.409
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.244
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.484
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.560
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.560
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo

INFO:root:Train Loss: 0.0497, AP_IoU_0.50_all: 0.7082


DONE (t=0.20s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.411
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.708
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.442
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.411
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.247
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.485
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.562
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.562
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
Epo